In [126]:
from itertools import combinations
from dataclasses import dataclass
import re
from typing import Text, List, Any, Generator
import pandas as pd
pd.set_option('display.max_columns', None)

import psycopg2
import psycopg2.extras

CONNECTION = psycopg2.connect(database="chembl_28", user="chembl", password="chembl123", host="127.0.0.1", port="5432")


COLUMNS = ('f_id', 'f_chembl',
           'h_id', 'h_chembl',
           'f_smiles',
           'h_smiles',
           'assay_id', 'assay_chembl',
           'assay_type',
           'target_id', 'target_chembl',
           'target_type',
           'f_type',
           'f_relation',
           'f_value',
           'f_units',
           'h_type',
           'h_relation',
           'h_value',
           'h_units',
           'result'
           )




Импортируем нужные нам библиотеки.
Называем колонки в нашей финальной таблице.

In [130]:
QUERY_GET_MOLECULES = """
    SELECT 
      md.MOLREGNO,
      md.CHEMBL_ID,
      cs.CANONICAL_SMILES,
      array_agg(a.ASSAY_ID) assays
    FROM MOLECULE_DICTIONARY AS md
    JOIN COMPOUND_STRUCTURES AS cs
      ON md.MOLREGNO=cs.MOLREGNO
    JOIN ACTIVITIES AS a
      ON md.MOLREGNO=a.MOLREGNO
    WHERE a.ASSAY_ID IS NOT NULL  /* сразу отбросим молекулы, по которым нет исследований */
    AND cs.CANONICAL_SMILES ~ 'F([^a-z]|$)'
    GROUP BY md.MOLREGNO, cs.MOLREGNO
    ORDER BY md.MOLREGNO
    {}
    ;
"""

def get_molecules_with_fluorine(offset=0, limit='ALL') -> List[Any]:
    query = QUERY_GET_MOLECULES.format(f'LIMIT {limit} OFFSET {offset}')
    
    cur = CONNECTION.cursor(cursor_factory=psycopg2.extras.NamedTupleCursor)
    cur.execute(query)
    rows = cur.fetchall()

    return rows


Функция для получения всех молекул с фтором, их ChEMBL_ID, SMILES, список assays). Сразу исключаем молекулы с фтором, по которым нет исследований.

In [87]:
molecules = get_molecules_with_fluorine()

In [135]:
len(molecules)

374882

In [127]:
QUERY_GET_MOLECULE_INFO = """
    SELECT 
      md.MOLREGNO AS f_id,
      md.CHEMBL_ID AS f_chembl,
      cs.CANONICAL_SMILES f_smiles,
      a.ASSAY_ID AS assay_id,
      a.CHEMBL_ID AS assay_chembl,
      a.ASSAY_TYPE AS assay_type,
      td.TID AS target_id,
      td.CHEMBL_ID AS target_chembl,
      td.TARGET_TYPE AS target_type,
      ac.STANDARD_TYPE AS f_type,
      ac.STANDARD_RELATION AS f_relation,
      ac.STANDARD_VALUE AS f_value,
      ac.STANDARD_UNITS AS f_units
    FROM MOLECULE_DICTIONARY AS md
    JOIN COMPOUND_STRUCTURES AS cs
      ON md.MOLREGNO = cs.MOLREGNO
    JOIN ACTIVITIES AS ac
      ON md.MOLREGNO = ac.MOLREGNO
    JOIN ASSAYS AS a
      ON ac.ASSAY_ID = a.ASSAY_ID
    JOIN TARGET_DICTIONARY AS td
      ON a.TID = td.TID
    WHERE cs.MOLREGNO = %s
    AND a.ASSAY_ID = %s
    AND ac.STANDARD_TYPE = %s
    ;
"""

def get_molecule_info(molregno, assay_id, standard_type):
    query = QUERY_GET_MOLECULE_INFO
    cur = CONNECTION.cursor(cursor_factory=psycopg2.extras.DictCursor)
    cur.execute(query, (molregno, assay_id, standard_type))
    row = cur.fetchone()

    return row

Получаем информацию о молекуле с фтором.

In [128]:
REGEX_FLUORINE = re.compile(r'(\(F\)|F(?![emlr])|F$)')

def get_all_fluorine_indexes(smiles):
    finds = list(REGEX_FLUORINE.finditer(smiles))
    return finds

In [ ]:
Получаем список всех вхождений F в SMILES молекулы с фтором.

In [129]:
def gen_next_smiles(finds: List[re.Match], smiles: Text) -> Generator[Text, None, None]:
    finds.reverse()
    for k in range(1, len(finds) + 1):
        for combination in combinations(finds, k):
            next_smiles = smiles
            for match in combination:
                next_smiles = next_smiles[:match.start()] + next_smiles[match.end():]
            yield next_smiles

Функция для генерации новых SMILES путем удаления F (замена F --> H), используются перестановки (число сочетаний из finds по k).

In [131]:
QUERY_GET_MOLECULES_BY_SMILES_AND_ASSAYS = """
    SELECT 
      md.MOLREGNO AS h_id,
      md.CHEMBL_ID AS h_chembl,
      cs.CANONICAL_SMILES h_smiles,
      a.ASSAY_ID AS assay_id,
      a.CHEMBL_ID AS assay_chembl,
      a.ASSAY_TYPE AS assay_type,
      td.TID AS target_id,
      td.CHEMBL_ID AS target_chembl,
      td.TARGET_TYPE AS target_type,
      ac.STANDARD_TYPE AS h_type,
      ac.STANDARD_RELATION AS h_relation,
      ac.STANDARD_VALUE AS h_value,
      ac.STANDARD_UNITS AS h_units
    FROM MOLECULE_DICTIONARY AS md
    JOIN COMPOUND_STRUCTURES AS cs
      ON md.MOLREGNO = cs.MOLREGNO
    JOIN ACTIVITIES AS ac
      ON md.MOLREGNO = ac.MOLREGNO
    JOIN ASSAYS AS a
      ON ac.ASSAY_ID = a.ASSAY_ID
    JOIN TARGET_DICTIONARY AS td
      ON a.TID = td.TID
    WHERE ac.ASSAY_ID IN %s
    AND cs.CANONICAL_SMILES = %s
    ;
"""

def get_molecule_by_smiles_where_in_assays(smiles: Text, assays: List[int]) -> List[Any]:
    query = QUERY_GET_MOLECULES_BY_SMILES_AND_ASSAYS
    cur = CONNECTION.cursor(cursor_factory=psycopg2.extras.DictCursor)
    cur.execute(query, (tuple(assays), smiles))
    molecule = cur.fetchall()

    return molecule

Функция для получения информации из assay о молекуле по переданному SMILES. 

In [132]:
def get_molecules_with_other_smiles(molecules: List[Any]) -> pd.DataFrame:
    molecules_info = pd.DataFrame(columns=COLUMNS)
    many_fluorine = []
    for i, molecule in enumerate(molecules):
        finds = get_all_fluorine_indexes(molecule.canonical_smiles) 
        if len(finds) > 12:
            print(i, len(molecules_info), len(many_fluorine), len(finds))
            many_fluorine.append(molecule)
            continue
        for other_smiles in gen_next_smiles(finds, molecule.canonical_smiles):
            other_molecules = get_molecule_by_smiles_where_in_assays(other_smiles, molecule.assays)
            for other_molecule in other_molecules:
                
                f_info = get_molecule_info(molecule.molregno, other_molecule['assay_id'], other_molecule['h_type'])
                if f_info:
                    
                    molecules_info = molecules_info.append(
                        pd.Series({**f_info, **other_molecule}, index=molecules_info.columns), ignore_index=True)

    return molecules_info, many_fluorine

Используем все ранее определенные функции. Помимо этого, если в молекуле больше 12 F, записываем ее в отдельный список (это сделано, потому что >2^12 комбинаций считается уже достаточно долго).    

In [133]:
import csv
def to_csv(obj, filename):
    with open(filename,'w') as out:
        csv_out = csv.writer(out)
        csv_out.writerow(obj[0]._fields)
        for row in obj:
            csv_out.writerow(row)

Функция для записи данных в .csv файл

In [137]:
def main(molecules, offset=0):
    batch_size = 1000
    
    i = 0
    for i in range(offset, int(len(molecules) / batch_size)):
        start = i*batch_size
        end = start + batch_size
        mol, many = get_molecules_with_other_smiles(molecules[start:end])
        mol.to_csv(f'mol_{start}_{end-1}.csv')
        print(f'saved mol_{start}_{end-1}.csv')
        if many:
            to_csv(many, f'many_mol_{start}_{end-1}.csv')                
    mol, many = get_molecules_with_other_smiles(molecules[i*batch_size:])
    mol.to_csv(f'mol_{i*batch_size}_{len(molecules)-1}.csv')
    if many:
        to_csv(many, f'many_mol_{i*batch_size}_{len(molecules)-1}.csv')       

Делим наши молекулы с фтором на "пакеты" по 1000 штук и записываем каждую тысячу в отдельный .csv файл (это сделано, чтобы если что-то пойдет не так, можно было остановить процесс и запустить заново с нужного места)

In [138]:
main(molecules)

115 110 0 21
116 110 1 18
saved mol_83000_83999.csv
290 254 0 19
291 254 1 18
292 254 2 18
293 254 3 17
294 254 4 16
307 256 5 13
761 892 6 18
saved mol_84000_84999.csv
907 837 0 13
908 837 1 15
909 837 2 17
925 837 3 13
926 837 4 17
928 837 5 13
929 837 6 17
saved mol_85000_85999.csv
saved mol_86000_86999.csv
saved mol_87000_87999.csv
463 542 0 17
545 613 1 36
546 613 2 24
547 613 3 36
548 613 4 36
549 613 5 36
550 613 6 60
551 613 7 102
552 613 8 156
saved mol_88000_88999.csv
286 130 0 17
287 130 1 17
288 130 2 13
294 130 3 17
295 130 4 13
saved mol_89000_89999.csv
466 544 0 13
467 544 1 13
706 897 2 17
saved mol_90000_90999.csv
saved mol_91000_91999.csv
saved mol_92000_92999.csv
788 923 0 13
789 923 1 17
790 923 2 13
791 923 3 17
792 923 4 21
804 926 5 13
805 926 6 13
806 926 7 13
807 926 8 13
saved mol_93000_93999.csv
saved mol_94000_94999.csv
saved mol_95000_95999.csv
saved mol_96000_96999.csv
saved mol_97000_97999.csv
saved mol_98000_98999.csv
saved mol_99000_99999.csv
saved mol_

saved mol_293000_293999.csv
629 352 0 17
saved mol_294000_294999.csv
saved mol_295000_295999.csv
73 28 0 16
950 545 1 16
saved mol_296000_296999.csv
saved mol_297000_297999.csv
saved mol_298000_298999.csv
saved mol_299000_299999.csv
172 68 0 15
saved mol_300000_300999.csv
saved mol_301000_301999.csv
saved mol_302000_302999.csv
saved mol_303000_303999.csv
saved mol_304000_304999.csv
saved mol_305000_305999.csv
saved mol_306000_306999.csv
saved mol_307000_307999.csv
saved mol_308000_308999.csv
saved mol_309000_309999.csv
saved mol_310000_310999.csv
saved mol_311000_311999.csv
saved mol_312000_312999.csv
saved mol_313000_313999.csv
saved mol_314000_314999.csv
saved mol_315000_315999.csv
saved mol_316000_316999.csv
713 377 0 24
saved mol_317000_317999.csv
saved mol_318000_318999.csv
620 751 0 13
saved mol_319000_319999.csv
876 1143 0 21
saved mol_320000_320999.csv
saved mol_321000_321999.csv
723 1003 0 18
857 1123 1 13
saved mol_322000_322999.csv
465 528 0 18
saved mol_323000_323999.csv
60